# Code Challenge - [Carrier Records Analysis]

Autor: Ignacio Rigoni  
Fecha: 21 de julio de 2025  
Contacto: ignacio.rigoni@gmail.com

## Summary

Try to identify where is most probable where a user is located based on data given a in carrier record's data file.



## 1. First step: Analyze CSV Data

The given [Data from 2/8/22](https://docs.google.com/spreadsheets/d/1yYEsD039_vLlGKzhpoJNM5nr6KvFj_O8sQWfi4v1RxE/edit?usp=sharing) will be manually analyzed in order to understand it and try to identify possible patterns.

After informing myself about how mobile cells work, I learned:
- Mobile devices do not connect to cell only by distance, but many other factors (obstacles, signal strength, distance, type of available network, load, etc)
- The given location belongs to the cell, not the device itself.

When analyzing data from 2/8/22, I wanted to know were the antennas where located, so I imported the list in "Google My Maps" obtaining this result:

[Map with Antennas](https://www.google.com/maps/d/edit?mid=16ZGM2jEMrieD6s3YfDStTMpv0sakfLo&usp=sharing)


After some analysis I deduced following conclusions:
- Empty rows must be ignored because they don't provide any useful data.
- It seems to be that there could be also "corrupted" rows (eg: latitude/longitude equal 0). Such rows should be also ignored.
- There is no information about the cell it self, except from the handled data type (data, voice, SMS) and its location (eg: we don't know the coverage area).
- It seems that location data (lon/lat) are exact for each antenna (I wasn't sure about this, that's why I made the map. Now I am).

## 2. Second Step: Clean Data

I want to continue examining data by cleaning the CSV files. This step means:
- Remove empty rows.
- Remove rows where there is no useful information (no antenne locations, etc)
- Remove rows where location is (0,0).

_Update:_
- Removes rows where there are no dates (not sure why this happens🤔)

In [ ]:
import pandas as pd

# ID de la hoja (extraída de la URL)
sheet_id = "1A8Z9dILpMYg_YqQQkgmi31ZhzJ4CBwvHBX3b4kzXE_Q"
sheet_name = "Page"

csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(csv_url).dropna(how="all")
df.columns = df.columns.str.strip()

# Convert to numeric (if it's a string)
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Filter nulls and zeros
df = df[(df['Latitude'].notna()) & (df['Longitude'].notna())]
df = df[(df['Latitude'] != 0) & (df['Longitude'] != 0)]

# Some dates are empty, lets filter them too.
df = df[(df['LocalDateTime'].notna()) & (df['UTCDateTime'].notna())]

Let's now see a little bit how does it looks like:

In [ ]:
df.head(5)

,Page,Item,UTCDateTime,LocalDateTime,Latitude,Longitude,TimeZone,City,County,State,Country,CellType
27,1,2,1/3/22 13:50,1/3/22 8:50,41.124750,-73.491694,America/New_York,New Canaan,Western Connecticut Planning Region,Connecticut,United States,Voice
28,15326,27,1/3/22 15:05,1/3/22 10:05,41.072392,-73.478188,America/New_York,Darien,Western Connecticut Planning Region,Connecticut,United States,SMS
31,1,3,1/3/22 15:17,1/3/22 10:17,40.856280,-73.522710,America/New_York,Oyster Bay,Nassau County,New York,United States,Voice
32,1,4,1/3/22 15:17,1/3/22 10:17,40.856280,-73.522710,America/New_York,Oyster Bay,Nassau County,New York,United States,Voice
33,1,5,1/3/22 15:17,1/3/22 10:17,40.856280,-73.522710,America/New_York,Oyster Bay,Nassau County,New York,United States,Voice


I will now save this to a new data sheet to be able to see clenaed data and manually analyze it.

In [ ]:
# Execute this ONLY ONCE. No need to do it every time.
import gspread
import pandas as pd
from google.colab import auth
import google.auth
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

In [ ]:
# Create and save the new cleaned up data.
sh = gc.create('CarrierDataCleaned')
sh.share(None, perm_type='anyone', role='writer')
worksheet = sh.get_worksheet(0)
set_with_dataframe(worksheet, df)

## 3. Third Step: Evalute a simple approach.

Now it's time to think about a simple ways to solve the problem with the information we have.

We have the following
- Variable _target_datetime_ represent the local date time at which we want to check where the subscriber was.
- Variable _window_ represent the window time (in minutes) before and after  _time_ to check carrier data.

### Method: Weighted Average by State

This is a very simple method.
Take the N entries from the data located inside the time window for the specified date time and count how many times each state is present.
The state with more occurrences (M) will be selected as the estimated state with a confidence of M/N.

In [ ]:
from datetime import datetime, timedelta

# target time
target_datetime = datetime.strptime("2/8/22 07:23", "%m/%d/%y %H:%M")
window = 15

df['LocalDateTime'] = pd.to_datetime(df['LocalDateTime'], errors='coerce')

# define window extremes
start_time = target_datetime - timedelta(minutes=window)
end_time = target_datetime + timedelta(minutes=window)

# get the 'windowed' entries
df_window = df[(df['LocalDateTime'] >= start_time) & (df['LocalDateTime'] <= end_time)]

# count values
if not df_window.empty and 'State' in df_window.columns:
    counts = df_window['State'].value_counts()
    top_state = counts.idxmax()
    confidence = counts.max() / len(df_window)

    print(f"Estimated state: {top_state}")
    print(f"Confidence: {confidence:.2%}")
else:
    print("No data available in the selected time window.")

Estimated state: Connecticut
Confidence: 100.00%
